# 誤差逆伝播法 

In [31]:
%matplotlib inline
import sys, os
sys.path.append("../deep-learning-from-scratch/")
import numpy as np
import matplotlib.pyplot as plt
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

# 乗算・加算レイヤによる順伝播・逆伝播の実装

In [20]:
#乗算レイヤ

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [21]:
#加算レイヤ

class  AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [22]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print("price = {}".format(int(price)))
print("dapple_num = {0}, dapple = {1}, dorange_num = {2}, dorange = {3}, dtax = {4}".format(\
                int(dapple_num), int(dapple), int(dorange_num), int(dorange), int(dtax)))

price = 715
dapple_num = 110, dapple = 2, dorange_num = 165, dorange = 3, dtax = 650


# 活性化関数レイヤの実装

In [23]:
#ReLUレイヤ
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [24]:
#Sigmoidレイヤ
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
            dx = dout * self.out * (1.0 - self.out)
            return dx

In [25]:
#Affineレイヤ
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [26]:
#Softmax-with-Lossレイヤ
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None    #損失
        self.y = None    #softmaxの出力
        self.t = None    #教師データ(one-hot vector)
        
    def softmax(x):
        c = np.max(x)
        exp_x = np.exp(x - c)    #オーバフロー対策
        sum_exp_x = np.sum(exp_x)
        y = exp_x / sum_exp_x
        return y
    
    def cross_entropy_error(self, y, t):
        delta = np.e**(-7)
        return -np.sum(t  * np.log(y + delta))
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

#  誤差逆伝播法の実装

In [27]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size,)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        #レイヤの生成
        self.layers = OrderedDict()
        self.layers['Affine1' ] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
       
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        #forward
        self.loss(x, t)
        
        #backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        #設定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [32]:
#勾配確認
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

b2:1.19682051769e-10
W1:2.21778466458e-13
W2:7.76842386351e-13
b1:9.85697693173e-13


# 誤差逆伝播法を用いた学習

In [33]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #誤差逆伝播法を用いて勾配を求める
    grad = network.gradient(x_batch, t_batch)
    
    #更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("training accuracy={0}, test accuracy={1}".format(train_acc, test_acc))

training accuracy=0.1181, test accuracy=0.1215
training accuracy=0.90505, test accuracy=0.9086
training accuracy=0.9218, test accuracy=0.9226
training accuracy=0.9366166666666667, test accuracy=0.9352
training accuracy=0.9439666666666666, test accuracy=0.9403
training accuracy=0.9497833333333333, test accuracy=0.9476
training accuracy=0.95665, test accuracy=0.953
training accuracy=0.95965, test accuracy=0.9562
training accuracy=0.9639166666666666, test accuracy=0.9608
training accuracy=0.967, test accuracy=0.9623
training accuracy=0.9688333333333333, test accuracy=0.9633
training accuracy=0.9711833333333333, test accuracy=0.9642
training accuracy=0.9734833333333334, test accuracy=0.9664
training accuracy=0.9735833333333334, test accuracy=0.9663
training accuracy=0.9751333333333333, test accuracy=0.9678
training accuracy=0.97685, test accuracy=0.9679
training accuracy=0.9789166666666667, test accuracy=0.9703
